# Chapter 4.9

## Importing Libraries

In [1]:
from __future__ import absolute_import, division, print_function
from __future__ import unicode_literals

In [2]:
import tensorflow as tf
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import matplotlib.pyplot as plt
import copy
import tqdm
from hfunc import models
from hfunc import metrics
import time

## Self-created functions

In [3]:
def estimate_node_importance(model, tester_model, layer_sizes, tol_low, tol_high, x, y, back=True, forward=True, single=False, layer=None):
    
    l, a = model.evaluate(x, y, verbose=0, batch_size=256)
    or_weights = model.get_weights()
    num_zeros, num_worse, num_important = (0, 0, 0)
    z = []
    wr = []
    imp = []
    amounts = []
    places = []
    if single:
        size = layer_sizes[layer]
        num_zeros, num_worse, num_important = (0, 0, 0)
        z = []
        wr = []
        imp = []
        for i in range(size):
            w = copy.deepcopy(or_weights)
            if back:
                w[2*layer][...,i] = 0
            w[2*layer+1][i] = 0
            if forward:
                w[2*layer+2][...,i,:] = 0
            tester_model.set_weights(w)
            nl, na = tester_model.evaluate(x, y, verbose=0, batch_size=256)
            change = l - nl
            if change <= tol_high and change >= tol_low:
                num_zeros += 1
                z += [i]
            elif change > 0:
                num_worse += 1
                wr += [i]
            else:
                num_important += 1
                imp += [i]
        amounts.append((num_zeros, num_worse, num_important))
        places.append((z, wr, imp))
        
        return amounts[0], places[0]
    else:
        for layer, size in enumerate(layer_sizes):
            num_zeros, num_worse, num_important = (0, 0, 0)
            z = []
            wr = []
            imp = []
            for i in range(size):
                w = copy.deepcopy(or_weights)
                if back:
                    w[2*layer][...,i] = 0
                w[2*layer+1][i] = 0
                if forward:
                    w[2*layer+2][...,i,:] = 0
                tester_model.set_weights(w)
                nl, na = tester_model.evaluate(x, y, verbose=0, batch_size=256)
                change = l - nl
                if change <= tol_high and change >= tol_low:
                    num_zeros += 1
                    z += [i]
                elif change > 0:
                    num_worse += 1
                    wr += [i]
                else:
                    num_important += 1
                    imp += [i]
            amounts.append((num_zeros, num_worse, num_important))
            places.append((z, wr, imp))
    
        return amounts, places

In [17]:
def optimize_weights(model, tester_model, layer_sizes, filter_sizes, tol_low, tol_high, x, y, input_size, output_size, min_imp_percentage=0.9, increasing=False, input_nodes=False):
    
    or_min_imp_percentage = min_imp_percentage
    
    for l, size in enumerate(layer_sizes):
        num_iter = 0
        min_imp_percentage = or_min_imp_percentage
        fil = filter_sizes[l]
        if l < len(layer_sizes) - 1:
            fil_out = filter_sizes[l+1]
        else:
            fil_out = None
        #print(fil_out)
        if l == 0:
            #print('First time')
            tmp_a, tmp_p = estimate_node_importance(model, tester_model, layer_sizes, low_tol, high_tol, x_val, y_val, layer=l, single=True)
        else:
            tmp_a, tmp_p = estimate_node_importance(model, tester_model, layer_sizes, low_tol, high_tol, x_val, y_val, back=False, layer=l, single=True)

        imp_z_ratio = (tmp_a[2] + tmp_a[0]) / layer_sizes[l]
        imp_ratio = (tmp_a[2]) / layer_sizes[l]
        #print(imp_z_ratio)
        #print(imp_ratio)
        #print('#####################')
        
        if l < len(layer_sizes) - 1:
            next_size = layer_sizes[l+1]
        else:
            next_size = output_size
            
        while imp_ratio <= min_imp_percentage:
            
            if num_iter == 50:
                min_imp_percentage -= 0.02
            
            w = model.get_weights()
            weight_len = len(w) - 3
            if fil:
                if fil_out:
                    limit2 = np.sqrt(6 / ((size*fil*fil)+(next_size*fil_out*fil_out)))
                else:
                    limit2 = np.sqrt(6 / ((size*fil*fil)+(next_size)))
            else:
                limit2 = np.sqrt(6 / ((size)+(next_size)))
            if l == 0:
                if input_nodes:
                    limit1 = np.sqrt(6 / (input_nodes+size))
                else:
                    limit1 = np.sqrt(6 / (input_size+size))
                

            if tmp_a[1]:
                
                if fil_out:
                    size_out = (fil_out, fil_out, tmp_a[1], next_size)
                else:
                    size_out = (tmp_a[1], next_size)
                
                w[2*l+2][..., tmp_p[1], :] = list(np.random.uniform(-limit2, limit2, size_out))
                if l == 0:
                    if fil:
                        size_in = (fil, fil, input_size, tmp_a[1])
                    else:
                        size_in = (input_size, tmp_a[1])
                    w[2*l][..., tmp_p[1]] = list(np.random.uniform(-limit1, limit1, size_in))

            if tmp_a[0]:
                
                if fil_out:
                    size_out = (fil_out, fil_out, tmp_a[0], next_size)
                else:
                    size_out = (tmp_a[0], next_size)
                    
                w[2*l+2][..., tmp_p[0], :] = list(np.random.uniform(-limit2, limit2, size_out))
                if l == 0:
                    if fil:
                        size_in = (fil, fil, input_size, tmp_a[0])
                    else:
                        size_in = (input_size, tmp_a[0])
                    w[2*l][..., tmp_p[0]] = list(np.random.uniform(-limit1, limit1, size_in))
            
            model.set_weights(w)
            if l == 0:
                tmp_a, tmp_p = estimate_node_importance(model, tester_model, layer_sizes, low_tol, high_tol, x_val, y_val, layer=l, single=True)
            else:
                tmp_a, tmp_p = estimate_node_importance(model, tester_model, layer_sizes, low_tol, high_tol, x_val, y_val, back=False, layer=l, single=True)
            imp_z_ratio = (tmp_a[2] + tmp_a[0]) / layer_sizes[l]
            imp_ratio = (tmp_a[2]) / layer_sizes[l]
            #print(imp_z_ratio)
            #print(imp_ratio)
            #print('#####################')
            if increasing:
                min_imp_percentage += increasing
            num_iter += 1

## Single-layer ANN

### MNIST

In [5]:
np.random.seed(2020)

In [6]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [7]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [8]:
tester_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [9]:
trials = 25
low_tol = -1e-5
high_tol = 1e-5
filter_sizes = [None]
layer_sizes = [128]
accs_opt = np.zeros((trials))
losses_opt = np.zeros((trials))
accs = np.zeros((trials))
losses = np.zeros((trials))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')
        ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    ) 
    
    w = model.get_weights()
    tester_model.set_weights(w)
    tester_model.fit(x_train, y_train, verbose=0, epochs=5)
    losses[trial], accs[trial] = tester_model.evaluate(x_test, y_test, verbose=0)
    
    optimize_weights(model, tester_model, layer_sizes, filter_sizes, low_tol, high_tol, x_val, y_val, 784, 10)
    
    model.fit(x_train, y_train, verbose=0, epochs=5)
    losses_opt[trial], accs_opt[trial] = model.evaluate(x_test, y_test, verbose=0)

  0%|                                                                                           | 0/25 [00:00<?, ?it/s]

None


  4%|███▎                                                                               | 1/25 [01:33<37:13, 93.07s/it]

None


  8%|██████▋                                                                            | 2/25 [02:39<32:34, 84.97s/it]

...

 92%|███████████████████████████████████████████████████████████████████████████▍      | 23/25 [31:22<02:49, 84.96s/it]

None


 96%|██████████████████████████████████████████████████████████████████████████████▋   | 24/25 [32:46<01:24, 84.43s/it]

None


100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [34:09<00:00, 81.99s/it]


In [15]:
A = pd.DataFrame(np.array([accs, accs_opt]).T, columns=['Unoptimized Weights', 'Optimized Weights'])
A.to_csv('../../../results/acc_ANN_iter_weights_mnist.csv')
A.describe()

,Unoptimized Weights,Optimized Weights
count,25.000000,25.000000
mean,0.975436,0.975128
std,0.001496,0.001529
min,0.972700,0.972400
25%,0.974300,0.973700
50%,0.975200,0.975600
75%,0.976800,0.976500
max,0.977400,0.977300


In [16]:
L = pd.DataFrame(np.array([losses, losses_opt]).T, columns=['Unoptimized Weights', 'Optimized Weights'])
L.to_csv('../../../results/loss_ANN_iter_weights_mnist.csv')
L.describe()

,Unoptimized Weights,Optimized Weights
count,25.000000,25.000000
mean,0.079142,0.081441
std,0.004107,0.005087
min,0.071206,0.073149
25%,0.076419,0.076339
50%,0.078328,0.081898
75%,0.081304,0.084012
max,0.086604,0.092926


### Fashion MNIST

In [23]:
np.random.seed(2020)

In [24]:
fmnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [25]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [26]:
tester_model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [27]:
trials = 25
low_tol = -1e-5
high_tol = 1e-5
filter_sizes = [None]
layer_sizes = [128]
accs_opt = np.zeros((trials))
losses_opt = np.zeros((trials))
accs = np.zeros((trials))
losses = np.zeros((trials))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
            tf.keras.layers.Flatten(input_shape=(28, 28)),
            tf.keras.layers.Dense(128, activation='relu'),
            tf.keras.layers.Dense(10, activation='softmax')
        ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    w = model.get_weights()
    tester_model.set_weights(w)
    tester_model.fit(x_train, y_train, verbose=0, epochs=5)
    losses[trial], accs[trial] = tester_model.evaluate(x_test, y_test, verbose=0)
    
    optimize_weights(model, tester_model, layer_sizes, filter_sizes, low_tol, high_tol, x_val, y_val, 784, 10)
    
    model.fit(x_train, y_train, verbose=0, epochs=5)
    losses_opt[trial], accs_opt[trial] = model.evaluate(x_test, y_test, verbose=0)

100%|██████████████████████████████████████████████████████████████████████████████████| 25/25 [37:18<00:00, 89.54s/it]


In [28]:
A = pd.DataFrame(np.array([accs, accs_opt]).T, columns=['Unoptimized Weights', 'Optimized Weights'])
A.to_csv('../../../results/acc_ANN_iter_weights_fmnist.csv')
A.describe()

,Unoptimized Weights,Optimized Weights
count,25.000000,25.000000
mean,0.868888,0.869064
std,0.006007,0.006144
min,0.857100,0.856200
25%,0.864300,0.866900
50%,0.869100,0.869800
75%,0.874500,0.873100
max,0.878900,0.878200


In [29]:
L = pd.DataFrame(np.array([losses, losses_opt]).T, columns=['Unoptimized Weights', 'Optimized Weights'])
L.to_csv('../../../results/loss_ANN_iter_weights_fmnist.csv')
L.describe()

,Unoptimized Weights,Optimized Weights
count,25.000000,25.000000
mean,0.364696,0.362768
std,0.014066,0.016294
min,0.342609,0.341724
25%,0.357195,0.350479
50%,0.361867,0.359648
75%,0.371032,0.366726
max,0.402705,0.406200


## Multi-layer Perceptron

### MNIST

In [5]:
np.random.seed(2020)

In [6]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [7]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [8]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [9]:
trials = 25
low_tol = -1e-5
high_tol = 1e-5
filter_sizes = [None, None, None]
layer_sizes = [128, 64, 32]
accs_opt = np.zeros((trials))
losses_opt = np.zeros((trials))
accs = np.zeros((trials))
losses = np.zeros((trials))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    w = model.get_weights()
    tester_model.set_weights(w)
    tester_model.fit(x_train, y_train, verbose=0, epochs=5)
    losses[trial], accs[trial] = tester_model.evaluate(x_test, y_test, verbose=0)
    
    optimize_weights(model, tester_model, layer_sizes, filter_sizes, low_tol, high_tol, x_val, y_val, 784, 10)
    
    model.fit(x_train, y_train, verbose=0, epochs=5)
    losses_opt[trial], accs_opt[trial] = model.evaluate(x_test, y_test, verbose=0)

100%|███████████████████████████████████████████████████████████████████████████████| 25/25 [1:56:08<00:00, 278.75s/it]


In [10]:
A = pd.DataFrame(np.array([accs, accs_opt]).T, columns=['Unoptimized Weights', 'Optimized Weights'])
A.to_csv('../../../results/acc_MLP_iter_weights_mnist.csv')
A.describe()

,Unoptimized Weights,Optimized Weights
count,25.000000,25.000000
mean,0.974096,0.973632
std,0.001981,0.002461
min,0.968500,0.967300
25%,0.973400,0.972600
50%,0.974300,0.974100
75%,0.975700,0.974800
max,0.977300,0.977800


In [11]:
L = pd.DataFrame(np.array([losses, losses_opt]).T, columns=['Unoptimized Weights', 'Optimized Weights'])
L.to_csv('../../../results/loss_MLP_iter_weights_mnist.csv')
L.describe()

,Unoptimized Weights,Optimized Weights
count,25.000000,25.000000
mean,0.090750,0.090004
std,0.007703,0.009228
min,0.077841,0.076328
25%,0.086862,0.084064
50%,0.090640,0.089402
75%,0.095472,0.093296
max,0.108661,0.117765


### Fashion MNIST

In [12]:
np.random.seed(2020)

In [13]:
fmnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [14]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [15]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Flatten(input_shape=(28, 28)),
    tf.keras.layers.Dense(128, activation='relu'),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(32, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [16]:
trials = 25
low_tol = -1e-5
high_tol = 1e-5
filter_sizes = [None, None, None]
layer_sizes = [128, 64, 32]
accs_opt = np.zeros((trials))
losses_opt = np.zeros((trials))
accs = np.zeros((trials))
losses = np.zeros((trials))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Flatten(input_shape=(28, 28)),
        tf.keras.layers.Dense(128, activation='relu'),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(32, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
    ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    w = model.get_weights()
    tester_model.set_weights(w)
    tester_model.fit(x_train, y_train, verbose=0, epochs=5)
    losses[trial], accs[trial] = tester_model.evaluate(x_test, y_test, verbose=0)
    
    optimize_weights(model, tester_model, layer_sizes, filter_sizes, low_tol, high_tol, x_val, y_val, 784, 10)
    
    model.fit(x_train, y_train, verbose=0, epochs=5)
    losses_opt[trial], accs_opt[trial] = model.evaluate(x_test, y_test, verbose=0)

100%|███████████████████████████████████████████████████████████████████████████████| 25/25 [3:37:47<00:00, 522.72s/it]


In [17]:
A = pd.DataFrame(np.array([accs, accs_opt]).T, columns=['Unoptimized Weights', 'Optimized Weights'])
A.to_csv('../../../results/acc_MLP_iter_weights_fmnist.csv')
A.describe()

,Unoptimized Weights,Optimized Weights
count,25.000000,25.000000
mean,0.868784,0.869532
std,0.004377,0.006400
min,0.859200,0.850500
25%,0.865500,0.867400
50%,0.869300,0.870500
75%,0.871600,0.874300
max,0.875900,0.878200


In [18]:
L = pd.DataFrame(np.array([losses, losses_opt]).T, columns=['Unoptimized Weights', 'Optimized Weights'])
L.to_csv('../../../results/loss_MLP_iter_weights_fmnist.csv')
L.describe()

,Unoptimized Weights,Optimized Weights
count,25.000000,25.000000
mean,0.367025,0.364099
std,0.011535,0.016962
min,0.347072,0.341189
25%,0.358815,0.355939
50%,0.366326,0.361810
75%,0.372493,0.368561
max,0.391896,0.426484


## Convolutional Neural Network

### MNIST

In [11]:
np.random.seed(2020)

In [12]:
mnist = tf.keras.datasets.mnist
(x_train, y_train), (x_test, y_test) = mnist.load_data()
x_train, x_test = x_train[..., np.newaxis], x_test[..., np.newaxis]
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [13]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [14]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [15]:
trials = 10
low_tol = -1e-5
high_tol = 1e-5
filter_sizes = [3, 3, 3, 3, None]
layer_sizes = [32, 64, 128, 256, 64]
accs_opt = np.zeros((trials))
losses_opt = np.zeros((trials))
accs = np.zeros((trials))
losses = np.zeros((trials))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
        ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    w = model.get_weights()
    tester_model.set_weights(w)
    tester_model.fit(x_train, y_train, verbose=0, epochs=5)
    losses[trial], accs[trial] = tester_model.evaluate(x_test, y_test, verbose=0)
    
    optimize_weights(model, tester_model, layer_sizes, filter_sizes, low_tol, high_tol, x_val, y_val, 1, 10, increasing=False, min_imp_percentage=0.6, input_nodes=784)
    
    model.fit(x_train, y_train, verbose=0, epochs=5)
    losses_opt[trial], accs_opt[trial] = model.evaluate(x_test, y_test, verbose=0)

100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [1:43:26<00:00, 620.69s/it]


In [16]:
A = pd.DataFrame(np.array([accs, accs_opt]).T, columns=['Unoptimized Weights', 'Optimized Weights'])
A.to_csv('../../../results/acc_CNN_iter_weights_mnist.csv')
A.describe()

,Unoptimized Weights,Optimized Weights
count,10.000000,10.000000
mean,0.990590,0.989870
std,0.001165,0.001374
min,0.988900,0.988100
25%,0.989600,0.988525
50%,0.990850,0.990100
75%,0.990900,0.990650
max,0.992700,0.991800


In [17]:
L = pd.DataFrame(np.array([losses, losses_opt]).T, columns=['Unoptimized Weights', 'Optimized Weights'])
L.to_csv('../../../results/loss_CNN_iter_weights_mnist.csv')
L.describe()

,Unoptimized Weights,Optimized Weights
count,10.000000,10.000000
mean,0.031711,0.037217
std,0.003131,0.005522
min,0.026262,0.027868
25%,0.029747,0.033312
50%,0.032108,0.037163
75%,0.033612,0.041179
max,0.037126,0.045922


### Fashion MNIST

In [5]:
np.random.seed(2020)

In [6]:
fmnist = tf.keras.datasets.fashion_mnist
(x_train, y_train), (x_test, y_test) = fmnist.load_data()
x_train, x_test = x_train[..., np.newaxis], x_test[..., np.newaxis]
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [7]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [8]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [9]:
trials = 10
low_tol = -1e-5
high_tol = 1e-5
filter_sizes = [3, 3, 3, 3, None]
layer_sizes = [32, 64, 128, 256, 64]
accs_opt = np.zeros((trials))
losses_opt = np.zeros((trials))
accs = np.zeros((trials))
losses = np.zeros((trials))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(28, 28, 1)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
        ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    w = model.get_weights()
    tester_model.set_weights(w)
    tester_model.fit(x_train, y_train, verbose=0, epochs=5)
    losses[trial], accs[trial] = tester_model.evaluate(x_test, y_test, verbose=0)
    
    optimize_weights(model, tester_model, layer_sizes, filter_sizes, low_tol, high_tol, x_val, y_val, 1, 10, increasing=False, min_imp_percentage=0.6, input_nodes=784)
    
    model.fit(x_train, y_train, verbose=0, epochs=5)
    losses_opt[trial], accs_opt[trial] = model.evaluate(x_test, y_test, verbose=0)

100%|███████████████████████████████████████████████████████████████████████████████| 10/10 [2:10:28<00:00, 782.83s/it]


In [10]:
A = pd.DataFrame(np.array([accs, accs_opt]).T, columns=['Unoptimized Weights', 'Optimized Weights'])
A.to_csv('../../../results/acc_CNN_iter_weights_fmnist.csv')
A.describe()

,Unoptimized Weights,Optimized Weights
count,10.000000,10.000000
mean,0.910950,0.910980
std,0.003020,0.002849
min,0.905000,0.906900
25%,0.908875,0.909150
50%,0.911200,0.910700
75%,0.913550,0.912475
max,0.914300,0.916800


In [11]:
L = pd.DataFrame(np.array([losses, losses_opt]).T, columns=['Unoptimized Weights', 'Optimized Weights'])
L.to_csv('../../../results/loss_CNN_iter_weights_fmnist.csv')
L.describe()

,Unoptimized Weights,Optimized Weights
count,10.000000,10.000000
mean,0.255865,0.255596
std,0.007882,0.006449
min,0.243792,0.246551
25%,0.249812,0.250484
50%,0.255222,0.256682
75%,0.263459,0.258626
max,0.266264,0.267190


### CIFAR 10

In [35]:
np.random.seed(2020)

In [36]:
cifar = tf.keras.datasets.cifar10
(x_train, y_train), (x_test, y_test) = cifar.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Converting interger values to floats (0 to 1)

In [37]:
x_train, x_val, y_train, y_val = train_test_split(x_train, y_train, train_size=0.85, stratify=None)

In [38]:
tester_model = tf.keras.models.Sequential([
    tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
    tf.keras.layers.MaxPool2D(),
    tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
    tf.keras.layers.Flatten(),
    tf.keras.layers.Dense(64, activation='relu'),
    tf.keras.layers.Dense(10, activation='softmax')
])
tester_model.compile(
    optimizer='adam',
    loss='sparse_categorical_crossentropy',
    metrics=['accuracy']
)

In [39]:
trials = 5
low_tol = -1e-5
high_tol = 1e-5
filter_sizes = [3, 3, 3, 3, None]
layer_sizes = [32, 64, 128, 256, 64]
accs_opt = np.zeros((trials))
losses_opt = np.zeros((trials))
accs = np.zeros((trials))
losses = np.zeros((trials))
for trial in tqdm.trange(trials):
    
    model = tf.keras.models.Sequential([
        tf.keras.layers.Conv2D(32, 3, padding='same', activation='relu', input_shape=(32, 32, 3)),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(64, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(128, 3, padding='same', activation='relu'),
        tf.keras.layers.MaxPool2D(),
        tf.keras.layers.Conv2D(256, 3, padding='same', activation='relu'),
        tf.keras.layers.Flatten(),
        tf.keras.layers.Dense(64, activation='relu'),
        tf.keras.layers.Dense(10, activation='softmax')
        ])
    model.compile(
        optimizer='adam',
        loss='sparse_categorical_crossentropy',
        metrics=['accuracy']
    )
    
    w = model.get_weights()
    tester_model.set_weights(w)
    tester_model.fit(x_train, y_train, verbose=0, epochs=5)
    losses[trial], accs[trial] = tester_model.evaluate(x_test, y_test, verbose=0)
    
    optimize_weights(model, tester_model, layer_sizes, filter_sizes, low_tol, high_tol, x_val, y_val, 3, 10, increasing=False, min_imp_percentage=0.4, input_nodes=3072)
    
    model.fit(x_train, y_train, verbose=0, epochs=5)
    losses_opt[trial], accs_opt[trial] = model.evaluate(x_test, y_test, verbose=0)

  0%|                                                                                            | 0/5 [14:55<?, ?it/s]


KeyboardInterrupt: 

In [ ]:
A = pd.DataFrame(np.array([accs, accs_opt]).T, columns=['Unoptimized Weights', 'Optimized Weights'])
A.to_csv('../../../results/acc_CNN_iter_weights_cifar.csv')
A.describe()

In [ ]:
L = pd.DataFrame(np.array([losses, losses_opt]).T, columns=['Unoptimized Weights', 'Optimized Weights'])
L.to_csv('../../../results/loss_CNN_iter_weights_cifar.csv')
L.describe()